In [1]:
import pandas as pd 
import numpy as np

In [2]:
dataset=pd.read_csv("Kenya_dataset1.csv")
dataset.head()

,Text,Category,Label
0,Is government gonna be taxed for it's idle lan...,Agriculture,9
1,I agree Gava should not talk of youth entrepre...,"Industry, Trade and Cooperatives",11
2,Huyu ameamua kuharibu barabara smh,Transport,2
3,Hakuna maji,Water and sanitation,6
4,Lack of water,Water and sanitation,6


In [3]:
dataset.tail()

,Text,Category,Label
4528,"""I'm attending webinars on water-efficient far...",Agriculture,9
4529,"""The drought has united farmers in our region ...",Agriculture,9
4530,"""I'm advocating for increased awareness of dro...",Agriculture,9
4531,"""The drought is a stark reminder of the import...",Agriculture,9
4532,"""Farmers are resilient and resourceful, findin...",Agriculture,9


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4533 entries, 0 to 4532
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      4533 non-null   object
 1   Category  4533 non-null   object
 2   Label     4533 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 106.4+ KB


In [5]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Label,4533.0,5.13148,3.073591,1.0,2.0,3.0,8.0,12.0


In [6]:
dataset.columns.values

array(['Text', 'Category', 'Label'], dtype=object)

Data PreProcessing

In [7]:
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
import string
import gensim
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
def pre_process(text):
    # Tokenization: Split the text into sentences and the sentences into words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    # Lowercase the words and remove punctuation
    words = [[word.lower() for word in sentence if word.isalnum()] for sentence in words]

    # Remove words with fewer than 3 characters
    words = [[word for word in sentence if len(word) > 2] for sentence in words]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [[word for word in sentence if word not in stop_words] for sentence in words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in words]

    # Stemming
    stemmer = PorterStemmer()
    words = [[stemmer.stem(word) for word in sentence] for sentence in words]

    return words




In [10]:
dataset['Text']= dataset['Text'].apply(pre_process)

In [11]:
dataset['Text'].head(10)

0    [[govern, gon, tax, idl, land], [watu, wapand,...
1    [[agre, gava, talk, youth, entrepreneurship, w...
2           [[huyu, ameamua, kuharibu, barabara, smh]]
3                                     [[hakuna, maji]]
4                                      [[lack, water]]
5                            [[pothol, kwa, barabara]]
6                           [[mashimo, kwa, barabara]]
7                                        [[bad, road]]
8                            [[stima, inapotea, sana]]
9    [[kplc, wanatucheza], [kila, siku, stima, hupo...
Name: Text, dtype: object

In [12]:
dataset['Text'].tail(10)

4523    [[particip, commun, program, distribut, relief...
4524      [[drought, forc, becom, adapt, resili, farmer]]
4525    [[explor, innov, technolog, sustain, farm, dro...
4526    [[lack, rainfal, made, appreci, beauti, sustai...
4527    [[farmer, share, stori, turn, advers, opportun...
4528    [[attend, webinar, farm, techniqu, adapt, chan...
4529    [[drought, unit, farmer, region, collect, addr...
4530    [[advoc, increas, awar, drought, impact, rural...
4531    [[drought, stark, remind, import, conserv, res...
4532    [[farmer, resili, resourc, find, way, thrive, ...
Name: Text, dtype: object

In [13]:
def sub_list(text):
    tokens = [word for sublist in text for word in sublist]
    return tokens

In [14]:
dataset['Processed_Text'] = dataset['Text'].apply(sub_list)

In [15]:
dataset['Processed_Text']

0       [govern, gon, tax, idl, land, watu, wapand, mi...
1       [agre, gava, talk, youth, entrepreneurship, wi...
2                [huyu, ameamua, kuharibu, barabara, smh]
3                                          [hakuna, maji]
4                                           [lack, water]
                              ...                        
4528    [attend, webinar, farm, techniqu, adapt, chang...
4529    [drought, unit, farmer, region, collect, addre...
4530    [advoc, increas, awar, drought, impact, rural,...
4531    [drought, stark, remind, import, conserv, resp...
4532    [farmer, resili, resourc, find, way, thrive, d...
Name: Processed_Text, Length: 4533, dtype: object

In [16]:
#counts of each category
category_counts = dataset['Category'].value_counts()
category_counts

Category
Transport                              1634
Broadcasting and telecommunications    1274
Agriculture                             755
Education                               737
Housing and urban development            38
Parks and Recreation                     28
Internal Security                        22
Medical Services                         21
Energy                                   18
Ethics and governance                     3
Water and sanitation                      2
Industry, Trade and Cooperatives          1
Name: count, dtype: int64

In [17]:
from gensim import corpora, models
 #Create a Dictionary and Corpus
dictionary = corpora.Dictionary(dataset['Processed_Text'])
corpus = [dictionary.doc2bow(text) for text in dataset['Processed_Text']]

# Train the LDA Model with 12 topics
lda_model = models.LdaModel(corpus, num_topics=12, id2word=dictionary, passes=15)

# Print Topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

# Assign Topics to Documents
dataset['Topic'] = dataset['Processed_Text'].apply(lambda text: lda_model[dictionary.doc2bow(text)][0][0])

# Display the DataFrame with assigned topics
print(dataset[['Text', 'Category', 'Topic']])








(0, '0.031*"organ" + 0.029*"farm" + 0.022*"control" + 0.018*"safaricom" + 0.016*"inform"')
(1, '0.025*"jinsi" + 0.022*"safi" + 0.016*"safaricom" + 0.014*"relief" + 0.012*"kufanya"')
(2, '0.208*"drought" + 0.021*"area" + 0.019*"plant" + 0.016*"consid" + 0.016*"affect"')
(3, '0.054*"kwa" + 0.043*"shamba" + 0.027*"hilu" + 0.022*"sana" + 0.021*"kilimo"')
(4, '0.072*"matatu" + 0.032*"bora" + 0.026*"ferri" + 0.026*"huduma" + 0.025*"practic"')
(5, '0.058*"crop" + 0.042*"farmer" + 0.018*"soil" + 0.016*"resili" + 0.016*"produc"')
(6, '0.130*"farm" + 0.033*"water" + 0.027*"due" + 0.018*"conserv" + 0.015*"safaricom"')
(7, '0.048*"airport" + 0.026*"safaricom" + 0.015*"get" + 0.015*"irrig" + 0.013*"work"')
(8, '0.027*"safaricom" + 0.016*"mazuri" + 0.014*"explor" + 0.011*"weather" + 0.010*"grow"')
(9, '0.077*"pothol" + 0.024*"safaricom" + 0.023*"local" + 0.020*"agricultur" + 0.017*"citi"')
(10, '0.060*"safaricom" + 0.023*"servic" + 0.021*"custom" + 0.018*"experi" + 0.015*"call"')
(11, '0.073*"school

In [18]:
dataset['Topic'] 

0       3
1       2
2       0
3       0
4       0
       ..
4528    0
4529    2
4530    0
4531    2
4532    2
Name: Topic, Length: 4533, dtype: int64